In [3]:
"""
Utility functions for working with an Inception v3 model, including functions to calculate Inception Score and FID.
"""
import numpy as np
import torch
import torch.nn.functional as F
from torchvision.models.inception import inception_v3
from scipy.linalg import sqrtm
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import Parameter as P

class WrapInception(nn.Module):
    def __init__(self, net):
        super(WrapInception,self).__init__()
        self.net = net
        self.mean = P(torch.tensor([0.485, 0.456, 0.406]).view(1, -1, 1, 1),
                      requires_grad=False)
        self.std = P(torch.tensor([0.229, 0.224, 0.225]).view(1, -1, 1, 1),
                     requires_grad=False)
    def forward(self, x):
        # Normalize x
        x = (x + 1.) / 2.0
        x = (x - self.mean) / self.std
        # Upsample if necessary
        if x.shape[2] != 299 or x.shape[3] != 299:
          x = F.interpolate(x, size=(299, 299), mode='bilinear', align_corners=True)
        # 299 x 299 x 3
        x = self.net.Conv2d_1a_3x3(x)
        # 149 x 149 x 32
        x = self.net.Conv2d_2a_3x3(x)
        # 147 x 147 x 32
        x = self.net.Conv2d_2b_3x3(x)
        # 147 x 147 x 64
        x = F.max_pool2d(x, kernel_size=3, stride=2)
        # 73 x 73 x 64
        x = self.net.Conv2d_3b_1x1(x)
        # 73 x 73 x 80
        x = self.net.Conv2d_4a_3x3(x)
        # 71 x 71 x 192
        x = F.max_pool2d(x, kernel_size=3, stride=2)
        # 35 x 35 x 192
        x = self.net.Mixed_5b(x)
        # 35 x 35 x 256
        x = self.net.Mixed_5c(x)
        # 35 x 35 x 288
        x = self.net.Mixed_5d(x)
        # 35 x 35 x 288
        x = self.net.Mixed_6a(x)
        # 17 x 17 x 768
        x = self.net.Mixed_6b(x)
        # 17 x 17 x 768
        x = self.net.Mixed_6c(x)
        # 17 x 17 x 768
        x = self.net.Mixed_6d(x)
        # 17 x 17 x 768
        x = self.net.Mixed_6e(x)
        # 17 x 17 x 768
        # 17 x 17 x 768
        x = self.net.Mixed_7a(x)
        # 8 x 8 x 1280
        x = self.net.Mixed_7b(x)
        # 8 x 8 x 2048
        x = self.net.Mixed_7c(x)
        # 8 x 8 x 2048
        pool = torch.mean(x.view(x.size(0), x.size(1), -1), 2)
        # 1 x 1 x 2048
        logits = self.net.fc(F.dropout(pool, training=False).view(pool.size(0), -1))
        # 1000 (num_classes)
        return pool, logits

# Load and wrap the Inception model
def load_inception_net(parallel=False):
    """Load and return the inception model."""
    inception_model = inception_v3(pretrained=True, transform_input=False)
    inception_model = WrapInception(inception_model.eval())
    return nn.DataParallel(inception_model) if parallel else inception_model

# Calculate Inception Score mean + std given softmax'd logits and number of splits
def calculate_inception_score(pred, num_splits=10):
    """Calculate and return the Inception Score."""
    scores = []
    for index in range(num_splits):
        part = pred[index * (len(pred) // num_splits):(index + 1) * (len(pred) // num_splits)]
        kl_div = part * (np.log(part) - np.log(np.mean(part, axis=0, keepdims=True)))
        scores.append(np.exp(np.mean(np.sum(kl_div, axis=1))))
    return np.mean(scores), np.std(scores)

In [4]:
from torch.utils.data import Dataset
import os
import numpy as np
from PIL import Image

class CelebAMaskDataset(Dataset):
    def __init__(self, dataroot, is_label=True, phase='train'):
        """
        Initializes the CelebAMaskDataset.

        Parameters:
            dataroot (str): The root directory where the data is stored.
            is_label (bool): Whether the dataset should load labeled data (True) or unlabeled (False).
            phase (str): The phase of the dataset to use ('train', 'val', 'test', or 'train-val').
        """
        self.dataroot = dataroot
        self.is_label = is_label
        self.phase = phase
        self.idx_list = self._load_index_list()

    def _load_index_list(self):
        """Load index list based on the phase, label requirement, and data root."""
        # Define the subdirectory based on whether data is labeled or not
#         data_root = os.path.join(self.dataroot, 'label_data' if self.is_label else 'unlabel_data')
        data_root = self.dataroot
        
        # Set the filename based on the dataset phase or the type of data
        if self.is_label:
            if self.phase == 'train':
                filename = 'train_full_list.txt'
            elif self.phase == 'val':
                filename = 'val_full_list.txt'
            elif self.phase == 'test':
                filename = 'test_list.txt'
            elif self.phase == 'train-val':
                self.tmp_data_path = data_root+ '/label_data'
                # Combine train and validation lists for labeled data
                train_list = self._load_file(self.tmp_data_path+'/train_full_list.txt')
                val_list = self._load_file(self.tmp_data_path+'/val_full_list.txt')
                return np.concatenate((train_list, val_list))
            else:
                raise ValueError("Invalid phase specified for labeled data.")
    
            # Handle the unlabeled data
            #filename = 'unlabel_list.txt'  # Default for all phases if unlabeled
        
        self.tmp_data_path = data_root+ '/unlabel_data'
        file_path = self.tmp_data_path + "/unlabel_list.txt"
        return self._load_file(file_path)

    def _load_file(self, file_path):
        """Helper function to load a file and raise an error if not found."""
#         print(file_path)
#         print(os.path.exists(file_path))
#         print(type(file_path))
        if not os.path.exists(file_path):
            raise FileNotFoundError(f"Expected file at {file_path} not found.")
        return np.loadtxt(file_path, dtype=str)

    def __len__(self):
        return len(self.idx_list)

    def __getitem__(self, idx):
        """Retrieve an item by its index."""
        img_path = self.tmp_data_path +'/images/' + self.idx_list[idx]
#         print("THE PATH OF IMAGE IS", img_path)
        img = Image.open(img_path).convert('RGB')
        return {'image': np.array(img)}


In [6]:
"""
This module computes Inception scores and other metrics for datasets using an Inception v3 model.
"""
import numpy as np
import torch
from torch.utils.data import DataLoader
import pickle
#import inception_utils
#from dataloader_1.dataset import CelebAMaskDataset
from torch.utils.data import ConcatDataset

@torch.no_grad()
def extract_features(loader, inception, device):
    """Extract features from the data loader using the inception model."""
    pools, logits = [], []
    #print("Entering Extract Feats!")
    for data in loader:
        img = data['image']
        img = img.permute(0,3,1,2).float()
        img = img.to(device)
#         img = data['image'].to(device)
        #print("<<<<<<<<<<<<<IMG SHAPE>>>>>>>>>>>>>>", img.shape)
        if img.shape[1] != 3:
            img = img.expand(-1, 3, -1, -1)  # Ensure 3 color channels
        #print("########## TYPE OF IMG ###########", type(img))
        pool_val, logits_val = inception(img)
#         logits_val= inception(img)
#         pools.append(pool_val.detach().cpu().numpy())
#         logits.append(torch.nn.functional.softmax(logits_val, dim=1).detach().cpu().numpy())
        logits.append(torch.nn.functional.softmax(logits_val, dim=1).detach().cpu().numpy())
        
    return np.concatenate(logits)

def get_dataset(path, dataset_name):
    """Load the labeled and unlabeled dataset based on the input arguments."""
    if dataset_name == 'celeba-mask':
        # Assuming that the labeled and unlabeled data are stored in the same root path but in different subdirectories
        labeled_dataset = CelebAMaskDataset( path, is_label=True, phase='train-val')
        unlabeled_dataset = CelebAMaskDataset( path, is_label=False)

#         # Optionally, you can load only labeled or only unlabeled depending on another argument or condition
#         if args.use_both_types:
            # Concatenate both datasets into a single dataset
        dataset = ConcatDataset([labeled_dataset, unlabeled_dataset])
#         else:
#             # Decide based on some condition or configuration
#             dataset = labeled_dataset if args.prefer_labeled else unlabeled_dataset
    else:
        raise Exception('No such dataset loader defined.')
    
    return dataset

def main():
    """Main function to compute Inception features and scores."""
    # Configuration parameters
    size = 1024
    batch = 500
    #n_sample = 50000
    output = 'output.pkl'
    image_mode = 'RGB'
    dataset_name = 'celeba-mask'
    path = 'C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ'

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    inception = load_inception_net()
    inception = inception.to(device)
    print(type(inception))
    
    dataset = get_dataset(path, dataset_name)
    #rint("Dataset Class Initiated")
    loader = DataLoader(dataset, batch_size=batch, num_workers=0)
    #rint("Dataloader Created ...")
#     logits = extract_features(loader, inception, device)
#     print(f'Extracted {logits.shape[0]} features')

#     IS_mean, IS_std = calculate_inception_score(logits)
#     print(f'Training data from dataloader has an Inception Score of {IS_mean:.5f} +/- {IS_std:.5f}')
#     print('Calculating means and covariances...')
#     mean, cov = np.mean(logits, axis=0), np.cov(logits, rowvar=False)

#     with open(output, 'wb') as f:
#         pickle.dump({'mean': mean, 'cov': cov, 'size': size, 'path': path}, f)
        
    pools, logits = extract_features(loader, inception, device)
    print(f'Extracted {pools.shape[0]} features')

    IS_mean, IS_std = inception_utils.calculate_inception_score(logits)
    print(f'Training data from dataloader has an Inception Score of {IS_mean:.5f} +/- {IS_std:.5f}')
    print('Calculating means and covariances...')
    mean, cov = np.mean(pools, axis=0), np.cov(pools, rowvar=False)

    with open(args.output, 'wb') as f:
        pickle.dump({'mean': mean, 'cov': cov, 'size': args.size, 'path': args.path}, f)


if __name__ == '__main__':
    main()


<class '__main__.WrapInception'>


In [73]:
import gc
torch.cuda.empty_cache()
gc.collect()

4043

In [ ]:
C:\Users\AGARW\Downloads\CelebAMask-HQ\CelebAMask-HQ\gghhhhhhhhh bgghgf unlabel_data
    dg

In [28]:
f = open("C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ/unlabel_data/unlabel_list.txt")
          C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ/unlabel_data/unlabel_list.txt'

In [29]:
dfhhos.path.exists("C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ/unlabel_data/unlabel_list.txt")


True

In [20]:
def process_image_masks(image_path, mask_list):
    # Read the image
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Initialize an empty mask with the same dimensions as the image, but with 3 color channels
    combined_mask = np.zeros(image.shape, np.uint8)

    # Process each mask
    for idx, mask_path in enumerate(mask_list):
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        mask = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)[1]
        color = color_map[idx + 1]  # Fetch color from the color_map based on the index

        # Apply color to regions with the mask
        for i in range(3):  # Assuming RGB
            combined_mask[:, :, i][mask == 255] = color[i]

    # Combine image and mask by superimposing
    combined_image = cv2.addWeighted(image, 0.7, combined_mask, 0.3, 0)

    return combined_image

In [148]:
file1 = open('C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ/label_data/val_full_list.txt', 'r')
Lines = file1.readlines()
image_paths =[]
mask_paths_list = []
i = 0
for line in Lines:
    count += 1
    s = line.strip()[:-4]
    image_paths.append('C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ/label_data/'+line.strip())
    a = 'C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ-mask-anno/14/'+s+'*.png'
    print(a)
    mask = glob.glob('C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ/CelebAMask-HQ-mask-anno/14/'+s+'*.png')
    mask.sort()
    mask_paths_list.append(mask)
    #print(mask)
    
    

C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ-mask-anno/14/29500*.png
C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ-mask-anno/14/29501*.png
C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ-mask-anno/14/29502*.png
C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ-mask-anno/14/29503*.png
C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ-mask-anno/14/29504*.png
C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ-mask-anno/14/29505*.png
C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ-mask-anno/14/29506*.png
C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ-mask-anno/14/29507*.png
C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ-mask-anno/14/29508*.png
C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ-mask-anno/14/29509*.png
C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ-mask-anno/14/29510*.png
C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ-mask-anno/14/29511*.png
C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ-mask-anno/14/29512*.png

C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ-mask-anno/14/29610*.png
C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ-mask-anno/14/29611*.png
C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ-mask-anno/14/29612*.png
C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ-mask-anno/14/29613*.png
C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ-mask-anno/14/29614*.png
C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ-mask-anno/14/29615*.png
C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ-mask-anno/14/29616*.png
C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ-mask-anno/14/29617*.png
C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ-mask-anno/14/29618*.png
C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ-mask-anno/14/29619*.png
C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ-mask-anno/14/29620*.png
C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ-mask-anno/14/29621*.png
C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ-mask-anno/14/29622*.png

C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ-mask-anno/14/29718*.png
C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ-mask-anno/14/29719*.png
C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ-mask-anno/14/29720*.png
C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ-mask-anno/14/29721*.png
C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ-mask-anno/14/29722*.png
C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ-mask-anno/14/29723*.png
C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ-mask-anno/14/29724*.png
C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ-mask-anno/14/29725*.png
C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ-mask-anno/14/29726*.png
C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ-mask-anno/14/29727*.png
C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ-mask-anno/14/29728*.png
C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ-mask-anno/14/29729*.png
C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ-mask-anno/14/29730*.png

C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ-mask-anno/14/29826*.png
C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ-mask-anno/14/29827*.png
C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ-mask-anno/14/29828*.png
C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ-mask-anno/14/29829*.png
C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ-mask-anno/14/29830*.png
C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ-mask-anno/14/29831*.png
C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ-mask-anno/14/29832*.png
C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ-mask-anno/14/29833*.png
C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ-mask-anno/14/29834*.png
C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ-mask-anno/14/29835*.png
C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ-mask-anno/14/29836*.png
C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ-mask-anno/14/29837*.png
C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ-mask-anno/14/29838*.png

C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ-mask-anno/14/29934*.png
C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ-mask-anno/14/29935*.png
C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ-mask-anno/14/29936*.png
C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ-mask-anno/14/29937*.png
C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ-mask-anno/14/29938*.png
C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ-mask-anno/14/29939*.png
C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ-mask-anno/14/29940*.png
C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ-mask-anno/14/29941*.png
C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ-mask-anno/14/29942*.png
C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ-mask-anno/14/29943*.png
C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ-mask-anno/14/29944*.png
C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ-mask-anno/14/29945*.png
C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ-mask-anno/14/29946*.png

In [149]:
print(len(image_paths), len(mask_paths_list))

500 500


In [150]:
mask_paths_list

[['C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ/CelebAMask-HQ-mask-anno/14\\29500_hair.png',
  'C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ/CelebAMask-HQ-mask-anno/14\\29500_l_brow.png',
  'C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ/CelebAMask-HQ-mask-anno/14\\29500_l_ear.png',
  'C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ/CelebAMask-HQ-mask-anno/14\\29500_l_eye.png',
  'C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ/CelebAMask-HQ-mask-anno/14\\29500_l_lip.png',
  'C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ/CelebAMask-HQ-mask-anno/14\\29500_neck.png',
  'C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ/CelebAMask-HQ-mask-anno/14\\29500_nose.png',
  'C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ/CelebAMask-HQ-mask-anno/14\\29500_r_brow.png',
  'C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ/CelebAMask-HQ-mask-anno/14\\29500_r_eye.png',
  'C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ/CelebAMask-HQ-mask-anno/14\

In [111]:
# def blend_masks(image_path, mask_paths):
#     image = cv2.imread(image_path)
#     if image is None:
#         raise FileNotFoundError(f"The image at {image_path} was not found.")
#     blended_mask = np.zeros(image.shape, np.uint8)  
    
#     for mask_path in mask_paths:
#         mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
#         if mask is None:
#             raise FileNotFoundError(f"The mask at {mask_path} was not found.")
#         mask = cv2.resize(mask, (image.shape[1], image.shape[0]), interpolation=cv2.INTER_NEAREST)
#         mask = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)[1]

#         mask_type = mask_path.split('_',1)[-1].split('.')[0]
#         color = get_color_for_mask(mask_type, color_map, color_palette)
        
#         color_mask = np.zeros(image.shape, np.uint8)
#         for i in range(3):
#             color_mask[:, :, i][mask == 255] = color[i]
#         blended_mask = cv2.addWeighted(blended_mask, 1.0, color_mask, 0.5, 0)  # Adjust opacity here

#     return blended_mask
def get_color_for_mask(mask_name, color_map, color_palette):
    if mask_name not in color_map:
        # Assign a new color from the palette, wrap around if palette is exhausted
        color_map[mask_name] = color_palette[len(color_map) % len(color_palette)]
    return color_map[mask_name]

# Initialize an empty dictionary to store color mappings
color_map = {}

In [145]:
color_map = {
    'cloth': (128, 0, 128),      # Magenta as the fallback color
    'hair': (0, 255, 0),         # Lime Green
    'mouth': (255, 165, 0),      # Orange
    'lip': (255, 165, 0),         # Indigo
    'neck': (255, 20, 147),      # Deep Pink
    'nose': (0, 255, 255),       # Cyan
    'skin': (75, 0, 130),       # Gold
    'brow': (0, 0, 255),         # Blue
    'eye': (255, 0, 0),        # Purple
    'ear': (255, 105, 180)       # Hot Pink
}
# Define the vibrant fallback and background color in RGB
background_color = (128, 0, 128)  # Purple

In [146]:
mask_priority = {
    'cloth': 1,    # Lowest priority
    'neck': 2,
    'skin': 3,
    'face': 4,
    'hair': 5,
    'ear': 6,
    'nose': 7,
    'mouth': 8,
    'lip': 9,
    'eye': 10,
    'brow': 11    # Highest priority
}

import cv2
import numpy as np

def blend_masks(image_path, mask_paths, background_color=(128, 0, 128)):
    image = cv2.imread(image_path)
    if image is None:
        raise FileNotFoundError(f"The image at {image_path} was not found.")

    # Initialize the blended mask with the background color
    blended_mask = np.full(image.shape, background_color, dtype=np.uint8)

    # Sort mask paths by priority, lowest first so high priority is applied last
    mask_paths = sorted(mask_paths, key=lambda x: mask_priority.get(x.split('_')[-1].split('.')[0], 0))

    for mask_path in mask_paths:
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        if mask is None:
            raise FileNotFoundError(f"The mask at {mask_path} was not found.")
        mask = cv2.resize(mask, (image.shape[1], image.shape[0]), interpolation=cv2.INTER_NEAREST)
        mask = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)[1]

        # Extract the mask type for color assignment
        mask_type = mask_path.split('_')[-1].split('.')[0]
        color = color_map.get(mask_type, background_color)  # Fallback to background color

        # Create a colored mask
        color_mask = np.zeros_like(image)
        for i in range(3):
            color_mask[:, :, i] = np.where(mask == 255, color[i], 0)

        # Blend this mask with the previously blended masks
        for i in range(3):
            blended_mask[:, :, i] = np.where(mask == 255, color_mask[:, :, i], blended_mask[:, :, i])

    return blended_mask

In [151]:
import matplotlib.pyplot as plt


for i, image_path in enumerate(image_paths):
    mask_image = blend_masks(image_path, mask_paths_list[i])
#     plt.figure(figsize=(10, 10))
#     plt.imshow(mask_image)
#     plt.title(f'Mask Only Image {i+1}')
#     plt.show()

    # Save the mask image
    save_path = image_path.replace('label_data/', 'label_data/label/') 
    cv2.imwrite(save_path, cv2.cvtColor(mask_image, cv2.COLOR_RGB2BGR))
    print(f"Saved blended mask image to {save_path}")


Saved blended mask image to C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ/label_data/label/29500.jpg
Saved blended mask image to C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ/label_data/label/29501.jpg
Saved blended mask image to C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ/label_data/label/29502.jpg
Saved blended mask image to C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ/label_data/label/29503.jpg
Saved blended mask image to C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ/label_data/label/29504.jpg
Saved blended mask image to C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ/label_data/label/29505.jpg
Saved blended mask image to C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ/label_data/label/29506.jpg
Saved blended mask image to C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ/label_data/label/29507.jpg
Saved blended mask image to C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ/label_data/label/29508.jpg
Saved blended mask image to 

Saved blended mask image to C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ/label_data/label/29576.jpg
Saved blended mask image to C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ/label_data/label/29577.jpg
Saved blended mask image to C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ/label_data/label/29578.jpg
Saved blended mask image to C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ/label_data/label/29579.jpg
Saved blended mask image to C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ/label_data/label/29580.jpg
Saved blended mask image to C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ/label_data/label/29581.jpg
Saved blended mask image to C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ/label_data/label/29582.jpg
Saved blended mask image to C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ/label_data/label/29583.jpg
Saved blended mask image to C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ/label_data/label/29584.jpg
Saved blended mask image to 

Saved blended mask image to C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ/label_data/label/29652.jpg
Saved blended mask image to C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ/label_data/label/29653.jpg
Saved blended mask image to C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ/label_data/label/29654.jpg
Saved blended mask image to C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ/label_data/label/29655.jpg
Saved blended mask image to C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ/label_data/label/29656.jpg
Saved blended mask image to C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ/label_data/label/29657.jpg
Saved blended mask image to C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ/label_data/label/29658.jpg
Saved blended mask image to C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ/label_data/label/29659.jpg
Saved blended mask image to C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ/label_data/label/29660.jpg
Saved blended mask image to 

Saved blended mask image to C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ/label_data/label/29728.jpg
Saved blended mask image to C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ/label_data/label/29729.jpg
Saved blended mask image to C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ/label_data/label/29730.jpg
Saved blended mask image to C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ/label_data/label/29731.jpg
Saved blended mask image to C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ/label_data/label/29732.jpg
Saved blended mask image to C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ/label_data/label/29733.jpg
Saved blended mask image to C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ/label_data/label/29734.jpg
Saved blended mask image to C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ/label_data/label/29735.jpg
Saved blended mask image to C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ/label_data/label/29736.jpg
Saved blended mask image to 

Saved blended mask image to C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ/label_data/label/29804.jpg
Saved blended mask image to C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ/label_data/label/29805.jpg
Saved blended mask image to C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ/label_data/label/29806.jpg
Saved blended mask image to C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ/label_data/label/29807.jpg
Saved blended mask image to C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ/label_data/label/29808.jpg
Saved blended mask image to C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ/label_data/label/29809.jpg
Saved blended mask image to C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ/label_data/label/29810.jpg
Saved blended mask image to C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ/label_data/label/29811.jpg
Saved blended mask image to C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ/label_data/label/29812.jpg
Saved blended mask image to 

Saved blended mask image to C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ/label_data/label/29880.jpg
Saved blended mask image to C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ/label_data/label/29881.jpg
Saved blended mask image to C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ/label_data/label/29882.jpg
Saved blended mask image to C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ/label_data/label/29883.jpg
Saved blended mask image to C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ/label_data/label/29884.jpg
Saved blended mask image to C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ/label_data/label/29885.jpg
Saved blended mask image to C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ/label_data/label/29886.jpg
Saved blended mask image to C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ/label_data/label/29887.jpg
Saved blended mask image to C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ/label_data/label/29888.jpg
Saved blended mask image to 

Saved blended mask image to C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ/label_data/label/29956.jpg
Saved blended mask image to C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ/label_data/label/29957.jpg
Saved blended mask image to C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ/label_data/label/29958.jpg
Saved blended mask image to C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ/label_data/label/29959.jpg
Saved blended mask image to C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ/label_data/label/29960.jpg
Saved blended mask image to C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ/label_data/label/29961.jpg
Saved blended mask image to C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ/label_data/label/29962.jpg
Saved blended mask image to C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ/label_data/label/29963.jpg
Saved blended mask image to C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ/label_data/label/29964.jpg
Saved blended mask image to 

In [120]:
 save_path = image_path.replace('label_data/', 'label_data/label/') 

In [121]:
save_path

'C:/Users/AGARW/Downloads/CelebAMask-HQ/CelebAMask-HQ/label_data/label/28000.jpg'